In [1]:
import numpy as np
import random as rn
import findspark
import sys
findspark.init("C:\\Users\\Software\\spark-3.3.1-bin-hadoop3\\bin\\..")
#findspark.init("D:\\Users\\software\\spark-3.2.1-bin-hadoop3.2\\bin\\..")

In [2]:
from pyspark import SparkContext
sc = SparkContext("local","PSO Distribuido")

In [3]:
def MSE(y, pred):
    n = len(y)
    if n != len(pred):
        print("error: datos y predicción de distintos tamaños")
        return -1

    resultado = 0.0

    for i in range(n):
        resultado += (y[i] - pred[i])**2

    resultado = resultado / n

    return resultado

In [4]:
V_max= 10
W= 1
c_1 = 0.3
c_2 = 0.7
objetivo =[50,50,50]

In [5]:
#Número dimensiones de los vectores
n=3
#Número de partículas
m= 10
#Número de iteraciones
I = 5

In [6]:

posiciones=[]
velocidades=[]
mejor_posiciones_locales=[]
mejor_pos_global= [0 for j in range(n)]
particulas = np.array([posiciones,velocidades,mejor_posiciones_locales])
best_local_fitness = []
#maximum float
max = sys.float_info.max
best_global_fitness = max

In [7]:
def InitParticles(part,N,M,bgf):
    for i in range(M):
        aux=[rn.uniform(-100,100) for j in range(N)]
        posiciones.append(aux)
        mejor_posiciones_locales.append(aux)
        aux=[rn.uniform(-100,100) for j in range(N)]
        velocidades.append(aux)
        fit = MSE(posiciones[i][:],objetivo)
        best_local_fitness.append(fit)
        if fit < bgf:
            bgf = fit
            mejor_pos_global = mejor_posiciones_locales[i][:]
    part = np.array([posiciones, velocidades, mejor_posiciones_locales],dtype=object)
    return bgf,mejor_pos_global,part 

In [8]:
def fitnessEval(part,mpl,mpg,blf,bgf, ind):
    filas=part[0:1,ind]
    fit = MSE(filas[0],objetivo)
    if fit< blf[ind]:
        blf[ind] = fit
        mpl[ind] = part[0:1,ind]
        if fit < bgf:
            bgf = fit
            mpg = part[0:1,ind]
    return part,mpl,mpg,blf,bgf

In [9]:
def posEval(part,mpl,mpg,ind):
    velocidades=particulas[1:2,ind][0]
    for k in range(n):
        velocidades[k] = W*velocidades[k] + c_1*r_1*(mejor_posiciones_locales[j][k] - particulas[0:1,j][0][k]) + c_2*r_2*(mejor_pos_global[k] - particulas[0:1,j][0][k])
        if velocidades[k] > V_max:
            velocidades[k] = V_max
        elif velocidades[k] < -V_max:
            velocidades[k] = -V_max  
        part[0:1,j][0][k] = part[0:1,j][0][k] + velocidades[k]
    return part,velocidades

In [10]:
best_global_fitness,mejor_pos_global,particulas = InitParticles(particulas,n,m,best_global_fitness)
for i in range(1):
    for j in range(m):
        particulas,mejor_posiciones_locales,mejor_pos_global,best_local_fitness,best_global_fitness=fitnessEval(particulas,mejor_posiciones_locales,mejor_pos_global,best_local_fitness,best_global_fitness,j)
    r_1 = rn.random()
    r_2 = rn.random()
    for j in range(m):
        particulas,velocidades = posEval(particulas,mejor_posiciones_locales,mejor_pos_global,j)
    print("Nuevas Posiciones->",particulas[0:1,j][0])

Nuevas Posiciones-> [-75.46606314216979 54.5761653752711 94.88716391601804]


In [11]:
#rdd_master= sc.parallelize(particulas)